# Chapter 5: Regression analysis

<b>Overview</b><br>
We use regression to uncover the correlation of multiple variables with the topics discussed in a review.<br>
The main questions are:<br>
<ul><li>How do school tests, poverty, and charter schools affect topics discussed in a review?</li>
<li>How do these relationships vary by state?</li></ul>


<b>Choice of model: Hierarchical Bayesian Multinomial Logit Regression</b><br>
We did not find LDA models that take covariates, and therefore preprocess the reviews data by estimating topics and then using regression to correlate these topics with independent of interest.  Ideally, we would like to estimate both the probability of a topic as a function of other variables in one go.<br>
The dependent variable is the topic discussed in a review, estimated from an LDA.  Topics vary from 0 to 1 across a number of topics; the sum of topics for any given review is 1.  Although the variable is not multinomial, its resemblence allows us to use multinomial logit regression to estimate how topic distribution changes with independent variables.<br>
The independent variables are:<br>
<ul><li>School tests, normalized across states</li>
<li>Poverty, as measured by the percent of students enrolled in free or discounted school lunches</li>
<li>Charter schools, relative to public schools.  Because private schools are not compelled to share test results or lunch program enrollments, we excluded them from this analysis.</li></ul>





<b>Assumptions</b><br>
Multinomial logit makes a number of assumptions that should be noted:
<ul><li>First, the independence of irrelevant alternatives assumption is a key weakness in the multinomial logit model.  This states that the risk ratio of two alternatives is independent of other alternatives in the set group.  In many settings, such as consumer choice between transportation options, this is not reasonable.  In our setting, we do not have a priori knowledge of how reviewers choose between topics to write.</li>
<li>Linearity - a usual assumption in many regressions</li>
<li>We assume coefficients do not change over time and are the same within a state.  We only use two years of review to guard against changes in effects across time</li></ul>

<b>Model</b><br>
We assume that the log odds of a topic relative to another topic is linear:<br>
$$log \frac{P(Y_{is}=k)}{P(Y_{is}=K)}=\beta_{0ks}+\beta_{1ks}X_{is}$$<br>
where $Y_{is}$ is the topic of review $i$ from state $s$.  Topics are indexed by $k=1,...,K$. Parameteres to be estimated are $\theta_{ks}=[\beta_{0ks},\beta_{1ks}]$.<br>
Then, the probability of any one option follows a multinomial logit form:<br>
$$P(Y_{is}=k)=\frac{exp{\beta_{0ks}+\beta_{1ks}X_{is}}}{\sum_{p=1}^K{\beta_{0ps}+\beta_{1ps}X_{is}}}$$<br>
We fix the an option to be 1 for identification, i.e., 
$$P(Y_{is}=K)=\frac{1}{1+\sum_{p=1}^{K-1}{\beta_{0ps}+\beta_{1ps}X_{is}}}$$<br>
In a usual multinomial situation where topics fell into one single topic, the likelihood is easy:<br>
$$log L(Y|\theta,X)=\sum_i \sum_k I(Y_{is}=k)\cdot log P(Y_{is}=k)$$<br>
However, our situation is a little bit messy, because we do not observe $Y_{is}$; we observe $Y_{isk}^{\prime}$,  which is continuous, bounded between 0 and 1, i.e, the probability that $Y_{is}$ falls in topic $k$, as estimated by LDA.  <br><br>
Still, we can amend the above likelihood to suit our needs.  Multinomial dictates that the allocation must fall into a single topic.  We allow the allocation to be spread across a number of topics.  Although LDA models the allocation process with a Dirichlet distribution, we abstract away from this for parsimony.<br><br>
Thus, our likelihood function is<br>
$$log L(Y|\theta,X)=\sum_i \sum_k{Y_{isk}^\prime\cdot log P(Y_{is}=k)}$$

<b>Estimation</b><br>
We estimate using a hierarchical Bayesian multinomial logit regression model.  Since multinomial logit estimate procedures typically require the independent variable to be categorical, we customize a procedure based on functions in the $R$ package $bayesm$.  We use Metropolis algorithm to draw from the posterior distribution:<br>
Namely, the procedure goes:<br>
<ul><li>Define priors parameters $\bar{\Delta},A,\nu,V$ (same as default in $rhierLinearModel$; see $bayesm$ document for notation)</li>
<li>Given $\theta_s^{old}$, the part of $theta$ relevant to state $s$ from the previous draw, draw a new candidate $\theta_s^{cand}$.</li>
<li>Calculate likelihood associated with $theta_s^{old}$ and $theta_s^{cand}$</li>
<li>Calculate the prior density associated with $theta_s^{old}$ and $theta_s^{cand}$</li>
<li>Accept as a function of the likelihood vlaues and prior densities.</li>
$$\alpha = \frac{L(Y|theta_s^{cand},X)p(\theta_s^{cand}|\Delta,\Sigma)}{L(Y|theta_s^{old},X)p(\theta_s^{old}|\Delta,\Sigma)} \\ 
P(accept)=min(\alpha,1)$$
<li>Once all states have been calculated, update $\Delta,\Sigma$ with Bayesian regression given $\theta$ and priors parameters $\bar{\Delta},A,\nu,V$</li>
<li>Iterate draws of $\theta=\{\theta_s\}$, $\Delta$, and $\Sigma$ until convergence.</li></ul>
Convergence was checked visually with plots.  MCMC was run for 60,000 iterations with burn rate of 30,000, keeping every 5 draws.  We calculate mean and 5% and 95% quantiles.

<b>Model fit</b><br>
Unfortunately, we did not have time to check the model.  Given more time, we would check the model against smaller more parsimonious models, for example, removing the hierarchy.<br>
We estimated an MLE version of the model with no hierarchy, but averaged at the school level instead of the review level due to data size issues.  The model converged and yielded the same significant population parameters as the hierarchical Bayesian model, but obviously could not detect state variation in correlations.<br>
What about overfitting?  Andrew Gelman wrote a blog (and has a working paper) about why he usually <a href="http://andrewgelman.com/2008/03/20/why_i_dont_usua_1/">does not worry about multiple testing</a>.  Bayesian regression handles this issue by using a prior to shrink the data towards the prior.  If the prior aligns with the "null", like centered around zero, then that helps guard against spurious correlations.

<b>Code</b><br>
The relevant R code are in the "rcode" folder:<br>
<ul><li>reg 1 mle regression.R: MLE estimate</li>
<li>reg 2 rhier regression - read data.R: Read data, make long data into wide data</li>
<li>reg 3 rhier regression - estimate.R: Estimate hierarchical Bayesian multinomial logit regression</li>
<li>rhierMultinomial.R: Function to estimate</li></ul>

In [3]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML
import urllib
thecode = open("rcode/reg 1 mle regression.R").read()
thehtml=highlight(thecode, PythonLexer(), HtmlFormatter())
print "### MLE code"
HTML(thehtml)

### MLE code


In [4]:
thecode = open("rcode/reg 2 rhier regression - read data.R").read()
thehtml=highlight(thecode, PythonLexer(), HtmlFormatter())
print "### Hierarchical Bayesian read data"
HTML(thehtml)

### Hierarchical Bayesian read data


In [5]:
thecode = open("rcode/reg 3 rhier regression - estimate.R").read()
thehtml=highlight(thecode, PythonLexer(), HtmlFormatter())
print "### Hierarchical Bayesian fit model"
HTML(thehtml)

### Hierarchical Bayesian fit model


In [6]:
thecode = open("rcode/rhierMultinomial.R").read()
thehtml=highlight(thecode, PythonLexer(), HtmlFormatter())
print "### Hierarchical Bayesian MCMC code"
HTML(thehtml)

### Hierarchical Bayesian MCMC code


<b>Model estimates</b><br>
We repeat the mean of the posterior distribution, zeroing out parameters that had 90% posterior interval covering 0, and coloring in red those that are significantly negative and in green those that are significantly positive.<br>
<img src="files/img_other_parameters.png"><br>
The coefficients can be interpreted as the change in log odds relative to an "other" topic for one unit change in the independent variable.<br>
We reestimated the parameters relative to "behavior", one of the presumably more negative topics to be discussed in a review.  We could estimate the model only once and recalculate log odds, but this should give the similar results, and we did not know which would be more insightful.  It turns out both are very insightful.<br>
<img src="files/img_behavior_parameters.png">


<b>Key insights</b><br>
<ul><li>School type greatly influences the topic of reviews.  Charter schools reviews are significantly more likely than public schools reviews to discuss student future, behaviors, and programs relative to other topics.  They are less likely to cover teacher and administration topics, as well as school culture than public schools.</li>
<li>Schools with better test scores are more likely to have reviews about student future and programs relative to behavior, but only in certain states (7 out of 14 states examined).</li>
<li>Poverty also steers the topics to more "behavior" subjects rather than student future, but only in six states (GA, MD, MI, NY, TX).

<b>Selected visuals</b>
Exponentiating the coefficients, we get the effect of a variable on the odds of a topic being mentioned in a review.  We find that charter schools have a significant impact on the conversations of behavior, future, and programs. (Public school probability normalized to 1)

<img src="files/img_charter_effect.png">

We find that poverty has an impact on conversations in only 5 out of 14 states.  In these states, schools with higher poverty, as measured by the % of free or discounted school lunches, reviews are half as likely to be about future of students than behavior issues.

<img src="files/img_poverty_state.png">